In [3]:
import csv
import pandas as pd
import pickle
import re
from sklearn.model_selection import train_test_split
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import numpy as np
import os.path
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_validate

In [4]:
data = pd.read_excel("Rest_Mex_2022.xlsx")
data.head(10)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
5,El peor huevos Benedict jamás,Hoy teníamos el desayuno por segunda vez en po...,1,Restaurant
6,Definitivamente no volvería a hospedarme en Sa...,"El hotel en si no es malo, pero mi experiencia...",1,Hotel
7,Terrible,No estoy seguro de por qué este restaurante ti...,1,Restaurant
8,"Bebidas ADULTERADAS, FATAL mi experiencia!!!",Llegué a este hotel “por desgracia” mi reserva...,1,Hotel
9,Hotel ha ido cuesta abajo,Hemos estado viniendo a Villa la Estancia dura...,1,Hotel


In [5]:
data['Opinion'][7693]

'Aunque me parece algo hipócrita el arte de Rivera (todo lo ve desde fuera desde una zona segura), debo reconocer que tiene algunas obras que me agradan. Este museo es interesante por el simple hecho de conocer el primer hogar que fuera de Rivera. Bastante fancy. Está chido si te gusta ver cosas antiguas.'

In [27]:
myTest = data.head(10000)
print(myTest)

                                      Title  \
0                              Pésimo lugar   
1                 No vayas a lugar de Eddie   
2              Mala relación calidad-precio   
3          Minusválido? ¡No te alojes aquí!   
4     Es una porqueria no pierdan su tiempo   
...                                     ...   
9995                               Vacacion   
9996                 Excelente y muy barato   
9997                      Mi lugar favorito   
9998                     Joysquad excelente   
9999               El hotel y sus empleados   

                                                Opinion  Polarity  Attraction  
0     Piensen dos veces antes de ir a este hotel, te...         1       Hotel  
1     Cuatro de nosotros fuimos recientemente a Eddi...         1  Restaurant  
2     seguiré corta y simple: limpieza\n- bad. Tengo...         1       Hotel  
3     Al reservar un hotel con multipropiedad Mayan ...         1       Hotel  
4     No pierdan su tiempo ni diner

# Normalizacion

In [24]:
nlp = spacy.load("es_core_news_sm")

In [25]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def lemmatize_token(token):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token)

# nltk:
def tokenizarNLTK(doc):
    if type(doc) != type(""):
        print(doc)
        return ""

    # Tokenize the document
    tokens = word_tokenize(doc, language='spanish')

    # Remove stop words and lemmatize
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [lemmatize_token(token) for token in tokens if token.lower() not in stop_words]

    # Join the filtered tokens into a string
    res = " ".join(filtered_tokens)

    return res

# spacy: 
def tokenizarSpacy(doc): 
    if(type(doc) != type("")):
         print(doc)
         return ""
    # nlp = spacy.load("es_core_news_sm") # español
    
    tkDoc = nlp(doc)
    res = ""
    for token in tkDoc:
        if(not(token.is_stop)): # quitando stop words
                res+=token.lemma_
                res+=" "

    return res

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
myTest['Opinion'] = myTest['Opinion'].apply(tokenizarSpacy)

KeyboardInterrupt: 

In [11]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen do veces ir hotel , molestan , hijo se...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro recientemente Eddie 's Place . servicio...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta simple : limpieza - bad . insist...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo dije...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"pierdan tiempo dinero , venimos boda aquí mism...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio primera ...",5,Hotel
9996,Excelente y muy barato,"Buena carne , tortilla recién hechas , queso f...",5,Restaurant
9997,Mi lugar favorito,"Siempre vamos , vamos seguido , comemos delici...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


In [12]:
myTest['Polarity'].value_counts()

Polarity
4    5878
3    2121
2     730
5     724
1     547
Name: count, dtype: int64

In [ ]:
result = myTest[myTest['Opinion']=="error"]

In [ ]:
result

,Title,Opinion,Polarity,Attraction


In [ ]:
myTest['Title'] = myTest['Title'].apply(tokenizar)

2016-12-01 00:00:00
2015-05-01 00:00:00


C:\Users\sergi\AppData\Local\Temp\ipykernel_21960\745785095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['Title'] = myTest['Title'].apply(tokenizar)


In [13]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen do veces ir hotel , molestan , hijo se...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro recientemente Eddie 's Place . servicio...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta simple : limpieza - bad . insist...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo dije...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"pierdan tiempo dinero , venimos boda aquí mism...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio primera ...",5,Hotel
9996,Excelente y muy barato,"Buena carne , tortilla recién hechas , queso f...",5,Restaurant
9997,Mi lugar favorito,"Siempre vamos , vamos seguido , comemos delici...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


# Splitting

In [14]:
train, test = train_test_split(myTest, test_size=0.2)

In [15]:
train

,Title,Opinion,Polarity,Attraction
5772,PRACTICO,"mejor sabor , ma económico menos rápido , prác...",4,Restaurant
7818,Lugar histórico,"Museografía interesante , información amplia h...",4,Attractive
7667,Muy buen hotel!!,"personal atento amigable , hacen sentir casa ,...",4,Hotel
2675,Agradable hotel - pero no acepte los bonos,Pasamos fin semana largo ( familia 4 ) agradab...,3,Hotel
4957,Muy buen desayuno buffet,"Buffet amplio mucha variedad , carnes , marisc...",4,Restaurant
...,...,...,...,...
2740,Pueden Mejorar,"museo bien , entradas $ 85 propina guía . malo...",3,Attractive
930,"Muy decepcionante. No es un 4 estrellas, apena...",Acabamos volver jueves semana Samba Vallarta ....,2,Hotel
2755,Más Leyenda,"hacer filum eterna subir escalón indicado , ro...",3,Attractive
4908,Magníficas carnes y una excelente relación cal...,"ir buena tienda carnicería , grill bistecs . a...",4,Restaurant


In [17]:
type(train['Opinion'])

pandas.core.series.Series

In [16]:
test

,Title,Opinion,Polarity,Attraction
2475,Bonito lugar pero no es justo para nosotros,Acabamos regresar estancia tres noches aquí . ...,3,Hotel
6520,buena opcion en familia,"platillos cena fin año parecieron deliciosos ,...",4,Hotel
1196,Mala atención a clientes,"punto turistico obligatorio , calidad comida b...",2,Attractive
3246,Un lugar a visitar,"lugar histórico , debes conocer visitas Guanaj...",3,Attractive
5118,Deliciosa comida mexicana.,"8 persona , lento servicio . guacamole viene e...",4,Restaurant
...,...,...,...,...
2100,Ambiente comida buena,buena experiencia musical vivo Sun Bar . Credi...,3,Hotel
7362,Bonito complejo... Personal amable,Pasamos noche Villa Palmar Flamingos después e...,4,Hotel
7811,Punto de interés en Guanajuato,"lugar puedes dejar visitar Guanajuato , acceso...",4,Attractive
9320,Increíble estancia en el Grand Luxxe Residence...,Acabo volver estancia semana Grand Luxxe Resid...,5,Hotel


# Text Representations

In [43]:
vectorizador_binario = CountVectorizer(binary=True, token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_frecuencia = CountVectorizer(token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_tfidf = TfidfVectorizer()

In [44]:
train_vectorizado = vectorizador_tfidf.fit_transform(train['Opinion'])

In [45]:
test_vectorizado = vectorizador_tfidf.transform(test['Opinion'])

In [46]:
print(vectorizador_tfidf.get_feature_names_out())
print(train_vectorizado.toarray())
print(len(train_vectorizado.toarray()[0]))
print(len(vectorizador_tfidf.get_feature_names_out()))

['00' '000' '00am' ... 'útiles' 'útlimo' 'ĺlegaron']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
29552
29552


# Using Sentiment lexicons

In [18]:
def load_sel():
	#~ global lexicon_sel
	lexicon_sel = {}
	input_file = open('SEL_full.txt', 'r', encoding="utf-8")
	for line in input_file:
		#Las líneas del lexicon tienen el siguiente formato:
		#abundancia	0	0	50	50	0.83	Alegría
		
		palabras = line.split("\t")
		palabras[6]= re.sub('\n', '', palabras[6])
		pair = (palabras[6], palabras[5])
		if lexicon_sel:
			if palabras[0] not in lexicon_sel:
				lista = [pair]
				lexicon_sel[palabras[0]] = lista
			else:
				lexicon_sel[palabras[0]].append (pair)
		else:
			lista = [pair]
			lexicon_sel[palabras[0]] = lista
	input_file.close()
	del lexicon_sel['Palabra']; #Esta llave se inserta porque es parte del encabezado del diccionario, por lo que se requiere eliminar
	#Estructura resultante
		#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	return lexicon_sel

In [19]:
if (os.path.exists('lexicon_sel.pkl')):
    lexicon_sel_file = open ('lexicon_sel.pkl','rb')
    lexicon_sel = pickle.load(lexicon_sel_file)
else:
    lexicon_sel = load_sel()
    lexicon_sel_file = open ('lexicon_sel.pkl','wb')
    pickle.dump(lexicon_sel, lexicon_sel_file)
    lexicon_sel_file.close()

In [20]:
print(lexicon_sel['gustar'])

[('Alegría', '0.596')]


In [21]:
def getSELFeatures(cadenas, lexicon_sel):
	#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	features = []
	for cadena in cadenas:
		valor_alegria = 0.0
		valor_enojo = 0.0
		valor_miedo = 0.0
		valor_repulsion = 0.0
		valor_sorpresa = 0.0
		valor_tristeza = 0.0
		cadena_palabras = re.split('\s+', cadena)
		dic = {}
		for palabra in cadena_palabras:
			if palabra in lexicon_sel:
				caracteristicas = lexicon_sel[palabra]
				for emocion, valor in caracteristicas:
					if emocion == 'Alegría':
						valor_alegria = valor_alegria + float(valor)
					elif emocion == 'Tristeza':
						valor_tristeza = valor_tristeza + float(valor)
					elif emocion == 'Enojo':
						valor_enojo = valor_enojo + float(valor)
					elif emocion == 'Repulsión':
						valor_repulsion = valor_repulsion + float(valor)
					elif emocion == 'Miedo':
						valor_miedo = valor_miedo + float(valor)
					elif emocion == 'Sorpresa':
						valor_sorpresa = valor_sorpresa + float(valor)
		dic['__txt__'] = cadena_palabras
		dic['__alegria__'] = valor_alegria
		dic['__tristeza__'] = valor_tristeza
		dic['__enojo__'] = valor_enojo
		dic['__repulsion__'] = valor_repulsion
		dic['__miedo__'] = valor_miedo
		dic['__sorpresa__'] = valor_sorpresa
		
		#Esto es para los valores acumulados del mapeo a positivo (alegría + sorpresa) y negativo (enojo + miedo + repulsión + tristeza)
		dic['acumuladopositivo'] = dic['__alegria__'] + dic['__sorpresa__']
		dic['acumuladonegative'] = dic['__enojo__'] + dic['__miedo__'] + dic['__repulsion__'] + dic['__tristeza__']
		
		features.append (dic)
	
	
	return features

In [22]:
train['Opinion']

5772    mejor sabor , ma económico menos rápido , prác...
7818    Museografía interesante , información amplia h...
7667    personal atento amigable , hacen sentir casa ,...
2675    Pasamos fin semana largo ( familia 4 ) agradab...
4957    Buffet amplio mucha variedad , carnes , marisc...
                              ...                        
2740    museo bien , entradas $ 85 propina guía . malo...
930     Acabamos volver jueves semana Samba Vallarta ....
2755    hacer filum eterna subir escalón indicado , ro...
4908    ir buena tienda carnicería , grill bistecs . a...
3601    Samba Vallarta veinte veinticuatro diciembre d...
Name: Opinion, Length: 8000, dtype: object

In [23]:
polaridad = getSELFeatures(train['Opinion'], lexicon_sel)
print("TEXTO")
print(train['Opinion'][7693])
print("POLARIDAD")
print (polaridad[0])

TEXTO
Aunque parece hipócrita arte Rivera ( ve zona segura ) , debo reconocer obras agradan . museo interesante simple hecho conocer primer hogar Rivera . Bastante fancy . chido si gusta ver cosas antigua .
POLARIDAD
{'__txt__': ['mejor', 'sabor', ',', 'ma', 'económico', 'menos', 'rápido', ',', 'práctico', 'cumple', 'objetivo', 'alimentarte', 'secas', ',', 'costo', 'ligeramente', 'elevado', '.'], '__alegria__': 0.0, '__tristeza__': 0.0, '__enojo__': 0.0, '__repulsion__': 0.0, '__miedo__': 0.0, '__sorpresa__': 0.0, 'acumuladopositivo': 0.0, 'acumuladonegative': 0.0}


In [ ]:
polaridad_cadena_pos = np.array([polaridad[0]['acumuladopositivo']])
polaridad_cadena_neg = np.array([polaridad[0]['acumuladonegative']])
polaridad_cadena = np.concatenate((polaridad_cadena_pos, polaridad_cadena_neg), axis=0)

# Training ML models

## Without NLTK

In [40]:
x_train = train_vectorizado
y_train = train['Polarity']
x_test = test_vectorizado
y_test = test['Polarity']

In [42]:
print(x_train)

  (0, 14655)	0.21489746316734312
  (0, 7944)	0.471997395738744
  (0, 21122)	0.12006246219586392
  (0, 3158)	0.1833612387058041
  (0, 4457)	0.12337146589146565
  (0, 1127)	0.28279096649142316
  (0, 15127)	0.17597894759636737
  (0, 20622)	0.29679247422734784
  (0, 13956)	0.21073375479499623
  (0, 13409)	0.1616876357544541
  (0, 22722)	0.28158176130441925
  (0, 26900)	0.19231032064514206
  (0, 25474)	0.10502001810462905
  (0, 25984)	0.186939071252508
  (0, 28322)	0.3550624111534327
  (0, 4464)	0.14299134621099757
  (0, 882)	0.27811785862837407
  (0, 17224)	0.10996194124670956
  (1, 16604)	0.03337324003299682
  (1, 3160)	0.04261507365136779
  (1, 16839)	0.029917570321517924
  (1, 22010)	0.04715152987918691
  (1, 28266)	0.031329232263059996
  (1, 6360)	0.04363701517231048
  (1, 6584)	0.04153125683630938
  :	:
  (7999, 17740)	0.08456088603468845
  (7999, 16980)	0.1006780001891303
  (7999, 18227)	0.17679878263876778
  (7999, 21529)	0.15026410661354023
  (7999, 18121)	0.07766753391186947
  (79

In [35]:
clf_polarity = LogisticRegression(max_iter=10000)
clf_polarity.fit(x_train,y_train)
y_pred = clf_polarity.predict(x_test)

In [17]:
clf_bayes = MultinomialNB()
clf_bayes.fit(x_train, y_train)
y_pred = clf_bayes.predict(x_test)

### Lematizacion, stop words. Binarizado. Logistic Regression:

In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Frecuencia. Logistic Regression:

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Logistic Regression:

In [46]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Multinomial Naive Bayes

In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## With NLTK

### Lematizacion, stop words. Binarizado. Logistic Regression:

In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.68      0.21      0.32       109
           2       0.44      0.05      0.09       143
           3       0.43      0.27      0.33       409
           4       0.69      0.95      0.80      1201
           5       0.74      0.10      0.18       138

    accuracy                           0.65      2000
   macro avg       0.59      0.32      0.34      2000
weighted avg       0.62      0.65      0.58      2000



### Lematizacion, stop words. Frecuencia. Logistic Regression:

In [36]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.68      0.21      0.32       109
           2       0.44      0.05      0.09       143
           3       0.43      0.27      0.33       409
           4       0.69      0.95      0.80      1201
           5       0.74      0.10      0.18       138

    accuracy                           0.65      2000
   macro avg       0.59      0.32      0.34      2000
weighted avg       0.62      0.65      0.58      2000



In [37]:
cv_results = cross_validate(clf_polarity, x_train,y_train, cv=5, scoring='f1_macro', verbose = 3, n_jobs = -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   20.7s remaining:   31.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   21.2s finished


In [38]:
cv_results['test_score'].mean()

0.27621456039405234